# Evaluation Script

This script is used for black-box and white-box evaluation of the trained models [substitute, base, adversarial training, Madry defense, superimposing and defensive distillation]. For black-box evaluation, the benchmark, the datasets generated with MI-FGSM ($\epsilon=[4,8,16,32]$) and the dataset contaminated with random noise ($\epsilon=8$) are used. For the white-box evaluation, MI-FGSM is applied to the target model itself.

## 0 - Dependencies

Import all necessary dependencies (Tensorflow, Keras, NumPy).

In [1]:
# Import NumPy
import numpy as np

# Tensorflow and Keras
import tensorflow as tf
import tensorflow.keras as keras

# Import momentum attack from cleverhans to perform white-box attacks
from attack_momentum import MomentumAttack

# Hide warnings
import warnings
warnings.filterwarnings('ignore')

## 1 - Load all Models

Load all the Tensorflow models to perform the evaluation.

In [ ]:
m_subsitute = tf.keras.models.load_model("trained_models/substitute.h5")
m_target = tf.keras.models.load_model("trained_models/target.h5")
m_at = tf.keras.models.load_model("trained_models/defenses/adversarial_training/adversarial_training.h5")
m_at_sampling = tf.keras.models.load_model("trained_models/defenses/adversarial_training/adversarial_training_with_sampling.h5")
m_mad_e8 = tf.keras.models.load_model("trained_models/defenses/madry/madry_e8_n10.h5")
m_mad_e16 = tf.keras.models.load_model("trained_models/defenses/madry/madry_e16_n20.h5")
m_si_l0 = tf.keras.models.load_model("trained_models/defenses/seiler/seiler_lambda0.h5")
m_si_l10 = tf.keras.models.load_model("trained_models/defenses/seiler/seiler_lambda10.h5")
m_DD = keras.models.load_model("trained_models/defenses/defensive_distillation/defensive_distillation.h5")

2022-07-22 13:55:53.766855: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-22 13:55:56.035988: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13745 MB memory:  -> device: 0, name: Quadro RTX 6000, pci bus id: 0000:d8:00.0, compute capability: 7.5


In [3]:
# more accessible structure
models = [("Substitute", m_subsitute), 
         ("Target", m_target),
         ("Adversarial Training", m_at),
         ("Adversarial Training with sampling", m_at_sampling),
         ("Madry e8", m_mad_e8),
         ("Madry e16", m_mad_e16),
         ("Superimposing l0", m_si_l0),
         ("Superimposing l10", m_si_l10),
         ("Defensive Distillation", m_DD)]

## 2 - Load all Datasets

Load all datasets for the evaluation.

In [4]:
# TinyImageNet validation dataset
validation_dataset = tf.data.experimental.load("datasets/tiny_imagenet/validation/")

# Clean evaluation dataset of 2500 examples correctly classified by the substitute
clean_dataset = tf.data.experimental.load("datasets/evaluation_clean/")

# With random noise peturbed dataset
noise_dataset = tf.data.experimental.load("datasets/evaluation_noise/")

# With MI-FGSM created adversarial pertubed dataset 
mi_fgsm_dataset_e4 = tf.data.experimental.load("datasets/adversarial_datasets/MI-FGSMAttack/e4/")
mi_fgsm_dataset_e8 = tf.data.experimental.load("datasets/adversarial_datasets/MI-FGSMAttack/e8/")
mi_fgsm_dataset_e16 = tf.data.experimental.load("datasets/adversarial_datasets/MI-FGSMAttack/e16/")
mi_fgsm_dataset_e32 = tf.data.experimental.load("datasets/adversarial_datasets/MI-FGSMAttack/e32/")

## 3 - Evaluation functions

Aid functions for the evaluation.

In [5]:
# Evaluate a given list of tensorflow models(name, model) on a given tensorflow dataset(ds) and print the accuracy and 
# name for each of the models
def evaluate(models, ds):
    for name, model in models:
        res = model.evaluate(ds, verbose=0)[1]*100
        print(name, " :", "{:0.1f}".format(res), "%")

In [6]:
# Perform an MI-FGSM white-box attack, similar to the black-box attack, on the target model itself.
# return the accuracy
def whitebox_attack(model, clean_ds):
    mi_fgsm = MomentumAttack()
    adv_ds = mi_fgsm.attack(clean_dataset, model, eps=8.0, eps_iter=0.8, nb_iter=20, norm=np.inf, decay_factor=1.0)
    adv_ds = adv_ds.batch(128)
    res = model.evaluate(adv_ds, verbose=0)
    res = res[1]*100
    return "{:0.1f}".format(res)

## 4 - Black-box Evaluation

Perform the black-box part of the evaluation:

**Validation Dataset** (time consuming without GPU)

In [7]:
print("Validation Accuracy")
print("---")
evaluate(models, validation_dataset)

Validation Accuracy
---


2022-07-18 12:41:38.005979: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8200


Substitute  : 72.0 %
Target  : 72.8 %
Adversarial Training  : 77.2 %
Adversarial Training with sampling  : 74.7 %
Madry e8  : 59.6 %
Madry e16  : 56.1 %
Superimposing l0  : 72.8 %
Superimposing l10  : 65.6 %
Defensive Distillation  : 76.2 %


---
**Clean Evaluation Dataset**

In [8]:
print("Clean Accuracy")
print("---")
evaluate(models, clean_dataset)

Clean Accuracy
---
Substitute  : 100.0 %
Target  : 87.4 %
Adversarial Training  : 92.4 %
Adversarial Training with sampling  : 91.2 %
Madry e8  : 73.7 %
Madry e16  : 70.9 %
Superimposing l0  : 89.4 %
Superimposing l10  : 80.4 %
Defensive Distillation  : 90.7 %


---
**Noise Dataset**

In [9]:
print("Noise Accuracy")
print("---")
evaluate(models, noise_dataset)

Noise Accuracy
---
Substitute  : 92.7 %
Target  : 85.6 %
Adversarial Training  : 79.4 %
Adversarial Training with sampling  : 73.7 %
Madry e8  : 73.7 %
Madry e16  : 66.6 %
Superimposing l0  : 86.6 %
Superimposing l10  : 79.6 %
Defensive Distillation  : 88.9 %


---
**MI-FGSM Datasets**

In [10]:
print("MI-FGSM black-box attack with epsilon=4 Accuracy")
print("---")
evaluate(models, mi_fgsm_dataset_e4)

MI-FGSM black-box attack with epsilon=4 Accuracy
---
Substitute  : 4.6 %
Target  : 20.8 %
Adversarial Training  : 35.8 %
Adversarial Training with sampling  : 66.6 %
Madry e8  : 71.3 %
Madry e16  : 68.1 %
Superimposing l0  : 15.4 %
Superimposing l10  : 57.1 %
Defensive Distillation  : 19.7 %


In [11]:
print("MI-FGSM black-box attack with epsilon=8 Accuracy")
print("---")
evaluate(models, mi_fgsm_dataset_e8)

MI-FGSM black-box attack with epsilon=8 Accuracy
---
Substitute  : 1.1 %
Target  : 6.8 %
Adversarial Training  : 23.0 %
Adversarial Training with sampling  : 65.2 %
Madry e8  : 69.2 %
Madry e16  : 66.1 %
Superimposing l0  : 4.5 %
Superimposing l10  : 39.9 %
Defensive Distillation  : 6.0 %


In [12]:
print("MI-FGSM black-box attack with epsilon=16 Accuracy")
print("---")
evaluate(models, mi_fgsm_dataset_e16)

MI-FGSM black-box attack with epsilon=16 Accuracy
---
Substitute  : 0.1 %
Target  : 2.9 %
Adversarial Training  : 50.4 %
Adversarial Training with sampling  : 59.7 %
Madry e8  : 63.9 %
Madry e16  : 58.7 %
Superimposing l0  : 1.8 %
Superimposing l10  : 22.0 %
Defensive Distillation  : 2.6 %


In [13]:
print("MI-FGSM black-box attack with epsilon=32 Accuracy")
print("---")
evaluate(models, mi_fgsm_dataset_e32)

MI-FGSM black-box attack with epsilon=32 Accuracy
---
Substitute  : 0.0 %
Target  : 1.6 %
Adversarial Training  : 50.2 %
Adversarial Training with sampling  : 38.9 %
Madry e8  : 50.7 %
Madry e16  : 46.4 %
Superimposing l0  : 1.1 %
Superimposing l10  : 11.1 %
Defensive Distillation  : 1.5 %


## 4 - White-box Evaluation

Perform the white-box part of the evaluation (time consuming without GPU):

In [14]:
print("MI-FGSM white-box attack with epsilon=8 Accuracy")
print("---")
for name, model in models:
    print(name, " : ", whitebox_attack(model, clean_dataset), "%")

MI-FGSM white-box attack with epsilon=8 Accuracy
---
Substitute  :  0.2 %
Target  :  7.1 %
Adversarial Training  :  4.9 %
Adversarial Training with sampling  :  6.9 %
Madry e8  :  26.9 %
Madry e16  :  30.2 %
Superimposing l0  :  6.0 %
Superimposing l10  :  8.8 %
Defensive Distillation  :  87.9 %


---
---

Succeeding step: Visualization of some results
**-> Results.ipynb**

Succeeding step: Analysis of results
**-> Thesis**